In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
import GP_training as gpt
sys.path.append("..")
import vvkernels as vvk
import sep_vvkernels as svvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
import numpy as np
import csv



%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
vf = VField()
f_target = vf.tgt_vec
sample_size = 100
D = vf.D
N = vf.N

def g_theta(sample_size, D):
    loc = np.random.random_sample((sample_size,D))
    return Tensor(loc)
train_x = g_theta(sample_size, D)
train_y = torch.zeros([sample_size, N])




for i in range(sample_size):
    train_y[i] = Tensor(vf(train_x[i]))# + torch.randn(Tensor(vf(train_x[i])).size()) * 0.005
    

In [3]:
def vfield_(x):
    x = x.reshape(x.shape[0],D)
    out = torch.zeros(x.shape[0], N)
    for i in range(x.shape[0]):
        out[i] = Tensor(vf(x[i])) #+ torch.randn(Tensor(vf(x[i])).size()) * 0.005
    return out

In [4]:
filename = 'datasets/data_train'
filename_test = 'datasets/data_val'
xx = train_x.numpy()
yy = train_y.numpy()
with open(filename, mode='w') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(sample_size):
        csv_writer.writerow([xx[i,0], xx[i,1], yy[i,0], yy[i,1]])
        
        
test_x = Tensor(np.random.random_sample((51,D)))
test_y = vfield_(test_x)
t_xx = test_x.numpy()
t_yy = test_y.numpy()
with open(filename_test, mode='w') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(51):
        csv_writer.writerow([t_xx[i,0], t_xx[i,1], t_yy[i,0], t_yy[i,1]])

In [5]:
def stopping_criteria(tol_vector, f_target, lower_bound, upper_bound):
    lower_tol_vector = f_target - tol_vector
    upper_tol_vector = f_target + tol_vector
    SUCCESS = True
    FAILURE = False
    for i in range(f_target.shape[0]):
            if (lower_bound[i] < lower_tol_vector[i]) or  (upper_bound[i] > upper_tol_vector[i]):
                SUCCESS = False  
            if ((lower_bound[i] > upper_tol_vector[i]) or  (upper_bound[i] < lower_tol_vector[i])):
                FAILURE = True
    return SUCCESS, FAILURE

In [6]:
class param_opti(nn.Module):
    def __init__(self, sample):
        super(param_opti, self).__init__()
        #loc = np.random.random_sample((loc_size,2))
        self.g_theta2 = nn.Parameter(Tensor(sample))
        
    def forward(self):
        
        return (self.g_theta2)

In [7]:
def conduct_param_opti(x,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_iter):
    model.eval()
    likelihood.eval()
    _par = param_opti(loc_sample)
    
    def closure():
        optimizer.zero_grad()
        g_theta2 = _par.forward()

        loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
        loss1 = -1 * loss1
        
        return loss1
    
    
    optimizer = torch.optim.LBFGS(_par.parameters(), lr=1., history_size=100, max_iter=100, line_search_fn="strong_wolfe")
    optimizer.step(closure)
    
    g_theta2 = _par.forward()

    loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
    loss1 = -1 * loss1
    
    print('Loss theta: %.3f' % ( loss1))
    
#     optimizer = torch.optim.Adam(_par.parameters(), lr=0.007)

# #     for j in range(training_iter):
# #         optimizer.zero_grad()
# #         g_theta2 = _par.forward()

# #         loss1, lower_bound, upper_bound = likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model, likelihood)
# #         loss1 = -1 * loss1
# #         loss1.backward(retain_graph=True)
# #         print('Iter %d/%d - Loss theta2: %.3f' % (j + 1, training_iter, loss1.item()))
# #         optimizer.step()
          
    return loss1, g_theta2, lower_bound, upper_bound

In [8]:
class design_opti(nn.Module):
    def __init__(self, _x):
        super(design_opti, self).__init__()
#         a = _x[0,0]
#         b = _x[0,1]
#         self.x_design_1 = nn.Parameter(a)
#         self.x_design_2 = nn.Parameter(b)
        self.x_design = nn.Parameter(_x)
    def forward(self):
#         x_design = torch.zeros(1,2)
#         x_design[0,0] = self.x_design_1
#         x_design[0,1] = self.x_design_2
        return (self.x_design)

In [9]:
def conduct_design_opti(x0,loc_sample, f_target, g_theta1, agg_data, model, likelihood, training_design_iter, training_param_iter, lr_new):
    design = design_opti(x0)
    loc_sample0 = loc_sample
    x_d = design.forward()
    
    def closure():
        optimizer.zero_grad()
        
        loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter)
        loss2.backward(retain_graph=True)
#         print(x_d)
#         print(lower_bound)
#         print(upper_bound)
       
        return loss2
        
        
        
    optimizer = torch.optim.LBFGS(design.parameters(), lr=lr_new, history_size=100, max_iter=100, line_search_fn="strong_wolfe")
    optimizer.step(closure)

    x_d = design.forward()
    loss2, g_theta2_out,lower_bound, upper_bound = conduct_param_opti(x_d,loc_sample, f_target,g_theta1, agg_data, model, likelihood, training_param_iter) ##takethis out
    print('Loss design: %.3f' % ( loss2))
    print(optimizer.state)
    return x_d, g_theta2_out, lower_bound, upper_bound
    
    

In [10]:
f_target = Tensor(vf.tgt_vec) 
f_target = f_target.reshape(f_target.shape[0],1) 
tol_vector = 0.0005 * torch.ones(f_target.shape)


loc_size = 10
loc_sample = np.random.random_sample((loc_size,2))
g_theta2_vec = (Tensor(loc_sample)).flatten()

filename = 'datasets/data_train'


x0 = Tensor(np.array([0.5,0.7])) 
x0 = x0.reshape(1,2)
x00 = x0 
vec_x = x00 


lr_new = 1.

SUCCESS = False 
FAILURE = False 
iter = 0 
tol = 0.009 
while(SUCCESS == False and iter < 50):
    print('START HYPERPARAMETERS optimization')
    model, likelihood = gpt.main()

    print('END HYPERPARAMETERS optimization')
    print(iter)
    
    x0_new,g_theta2,lower_bound, upper_bound = conduct_design_opti(x0, loc_sample, f_target, g_theta1, agg_data, model, likelihood, iter_design,iter_param, lr_new)
    g_theta2_detach = g_theta2.detach()
    xx  = g_theta2_detach.numpy()
    new_data = vfield_(g_theta2_detach)
    yy = new_data.numpy()
    with open(filename, mode='w') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i in range(g_theta1.shape[0], g_theta1.shape[0] + loc_size):
            csv_writer.writerow([xx[i,0], xx[i,1], yy[i,0], yy[i,1]])
    
        
    
    g_theta2_vec = torch.cat([g_theta2_vec, g_theta2.flatten()], 0)

    print(lower_bound)
    print(upper_bound)
    print(f_target-tol_vector)
    print(f_target+tol_vector)
    loc_sample = np.random.random_sample((loc_size,2))
    x0 = x0_new #Tensor(np.random.random_sample((1,2))) #x0_new
    vec_x = torch.cat([vec_x, x0_new])

    SUCCESS, FAILURE = stopping_criteria(tol_vector, f_target, lower_bound, upper_bound)
    iter = iter + 1
    print(x0_new)

File descriptor limit 256 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'


START HYPERPARAMETERS optimization


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-09-20 12:09:08,536	WARNING experiment.py:296 -- No name detected on trainable. Using DEFAULT.
2021-09-20 12:09:08,537	INFO registry.py:67 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 18.0/32.0 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/16 CPUs, 0/0 GPUs, 0.0/11.89 GiB heap, 0.0/5.94 GiB objects
Result logdir: /Users/mngom/ray_results/DEFAULT_2021-09-20_12-09-08
Number of trials: 17/30 (16 PENDING, 1 RUNNING)
+---------------------+----------+-------+----------+-------------+
| Trial name          | status   | loc   |   num_bk |          lr |
|---------------------+----------+-------+----------+-------------|
| DEFAULT_77b25_00000 | RUNNING  |       |        9 | 0.0276126   |
| DEFAULT_77b25_00001 | PENDING  |       |        4 | 0.000134408 |
| DEFAULT_77b25_00002 | PENDING  |       |        5 | 0.136756    |
| DEFAULT_77b25_00003 | PENDING  |       |        2 | 0.00288554  |
| DEFAULT_77b25_00004 | PENDING  |       |        3 | 0.00506157  |
| DEFAULT_77b25_00005 | PENDING  |       |        9 | 0.00121609  |
| DEFAULT_77b2

Result for DEFAULT_77b25_00000:
  date: 2021-09-20_12-09-11
  done: true
  experiment_id: 1b5da345bd2f4028af5f8d1e9dfca36b
  hostname: MacBook-Pro.local
  iterations_since_restore: 2
  loss: tensor(0.9026)
  node_ip: 192.168.0.11
  pid: 35825
  should_checkpoint: true
  time_since_restore: 0.20698213577270508
  time_this_iter_s: 0.09944605827331543
  time_total_s: 0.20698213577270508
  timestamp: 1632157751
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: 77b25_00000
  
Result for DEFAULT_77b25_00011:
  date: 2021-09-20_12-09-11
  done: true
  experiment_id: 53cc225f39d144d7b21a075b127a29b8
  hostname: MacBook-Pro.local
  iterations_since_restore: 1
  loss: tensor(0.8760)
  node_ip: 192.168.0.11
  pid: 35815
  should_checkpoint: true
  time_since_restore: 0.05167579650878906
  time_this_iter_s: 0.05167579650878906
  time_total_s: 0.05167579650878906
  timestamp: 1632157751
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 77b25_00011
  
Result for DEFAULT

Result for DEFAULT_77b25_00021:
  date: 2021-09-20_12-09-14
  done: true
  experiment_id: 5118c528f85b4342a24ddc43e6a09efd
  hostname: MacBook-Pro.local
  iterations_since_restore: 4
  loss: tensor(0.8475)
  node_ip: 192.168.0.11
  pid: 35871
  should_checkpoint: true
  time_since_restore: 0.12717700004577637
  time_this_iter_s: 0.02314901351928711
  time_total_s: 0.12717700004577637
  timestamp: 1632157754
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: 77b25_00021
  
Result for DEFAULT_77b25_00023:
  date: 2021-09-20_12-09-14
  done: false
  experiment_id: fb0c74ec74bb4fa6815bf1d5135869f8
  hostname: MacBook-Pro.local
  iterations_since_restore: 1
  loss: tensor(0.8703)
  node_ip: 192.168.0.11
  pid: 35879
  should_checkpoint: true
  time_since_restore: 0.056920766830444336
  time_this_iter_s: 0.056920766830444336
  time_total_s: 0.056920766830444336
  timestamp: 1632157754
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 77b25_00023
  
Result for DEF

2021-09-20 12:09:14,922	INFO tune.py:561 -- Total run time: 6.39 seconds (6.09 seconds for the tuning loop).
2021-09-20 12:09:15,025	WARNING experiment_analysis.py:645 -- Could not find best trial. Did you pass the correct `metric` parameter?


None


AttributeError: 'NoneType' object has no attribute 'config'

In [ ]:
#x0 = Tensor(np.array([0.1937, 0.1257]))
#x0 = Tensor(np.array([0.1885, 0.1038]))

x0 = Tensor(np.array([0.2016, 0.0966]))
print(vf(x0))
x0 = x0.reshape(1,2)
print(x0)
model.eval()
likelihood.eval()

pr = likelihood(model(x0))
print(pr.mean)

In [ ]:
iter

In [ ]:
res = 0
res = res + torch.ones(10,10)

In [ ]:
print(g_theta2_vec.shape)

In [ ]:
v2 = g_theta2_vec.reshape(70,2)
print(vec_x)

In [ ]:
print(v2)
np.savetxt('g_theta2_005.txt', v2.detach().numpy())

In [ ]:

np.savetxt('vec_x_005.txt', vec_x.detach().numpy())

In [ ]:
base_kernels = []
for i in range(4):
    base_kernels.append(vvk_rbf.vvkRBFKernel())

In [ ]:
base_kernels

In [ ]:
np.isnan(train_x)